# Agenda, week 5

1. Q&A
2. What are modules? What do they give us?
3. The various forms of the `import` statement
4. Developing a simple module
5. What happens when we `import` a module -- the special `__name__` variable
6. Python's standard library
7. PyPI and `pip` -- installing third-party modules
8. A little bit about `uv`
9. Using third-party modules -- how do you find, choose, download, and use them (and do that safely)?
10. AMA -- ask me anything
11. What next? Now that you've finished this course, where do you go for additional Python learning, resources, etc?

# What are modules? What do they give us?

We've talked a number of times about the "Don't repeat yourself" rule in programming, aka "DRY." If you have written something once, you shouldn't repeat it!

1. If the same line is repeated several times in a row, we should replace that with a loop.
2. If the same code is repeated several times in a program, we should replace that with a function.
3. If the same code is repeated several times across multiple programs, we can use a *library*.

A library is a term used in many programming languages to describe either code or data that was defined by someone else, in the past, which you can load into ("borrow," if you will) your program. You don't have to reinvent the wheel this way!

Just about every programming language supports libraries. This way, you can save yourself time in the future, or you can save your colleagues time, or they can save you time.

Moreover, if there is a bug in a library, it can be fixed once and affect all of our programs. Or if there's an optimization/fix/improvement, then we can fix it once, and affect all of our programs.

In Python, we call our libraries "modules" and "packages":

- A module is a single file in which we've defined Python functions and/or variables
- A package is a directory containing one or more modules.

When we use a module in Python, we get to reuse code and data that other people have defined.

A module also defines what we call a "namespace," a way to ensure that variable names are separated from one another, so different pieces of your program don't step on each other's variables.

# AP: What do you mean by "separate namespaces" for data and functions?

If I have a software product called ABCDE, and I take out a trademark on that product, then no one else in the software industry can have a product called ABCDE, because that would be confusing.

But if someone comes out with a car and calls it ABCDE, then I cannot stop them from having that trademark, because it's a different field.

You could say that even though the name is the same, it's in two different "namespaces," it has two different domains.

Another example: My name is "Reuven," and there are other Reuvens around. How do we distinguish between me and the others? We use last names! Last names serve as a "namespace," ensuring that I am easily distinguished from other Reuvens out there.

The problem of namespacing is basically that we need to resolve the ambiguity when there are multiple thigns with the same name.

In many languages, the namespace for functions and the namespace for variables is kept separate. You can have variables `a`, `b`, and `c`, and these names have nothing whatsoever to do with the functions called `a`, `b`, and `c`. They will never interfere with one another. (Whether this is a good idea for you to do in your program is separate.)

Python doesn't make this distinction. In Python, we have a single namespace, a single last name, a single domain for trademarks, for all of the things we define, both functions and variables. 

- If you define a variable `print`, you have just erased (in some ways) the builtin function `print`.   Only one can exist at a time! (This isn't quite true in the case of `print` and other builtin functions, but it will feel that way if you make such a definition!)
- If you define a function `total` that takes a bunch of numbers and returns their total, and then you invoke the function and assign the result to a variable named `total`, you have now erased the function's definition! Because now `total` is an integer, not a function.

Among other things, this means that you have to be a bit careful about what names you give to variables and functions, to ensure that they don't collide in this sort of way.

When we use `def` to define a function, we're really assigning to a variable. 

# How do we use a module in Python?

Python comes with a large collection of modules known as the "standard library." One of the modules in there is `random`, which contains a large number of functions having to do with random values -- getting random integers, choosing random elements of a list, etc.

If we want to use that functionality (rather than invent it ourselves), we need to tell Python that we want to load up the `random` module. How do we do that?

We use the `import` statement. It looks like this:

In [1]:
import random

# Dive into the syntax of `import`

1. We say `import`. Note that it's not a function! We don't put the name to its right in `()`.
2. The name to its right, unlike *most* programming languages I've used in the last, is not a string indicating the filename that we want to import or read. Rather, it's the name of the variable into which the module's values will be assigned.
3. There is no (easy, standard) way to use `import` with the name of a file. Rather, you provide the name of the module, and Python uses that as the basis for finding and loading the module you named.

Now that I've imported `random`, what can I do with it? 

Answer: Use any/all of the values that it now contains.

Well, what does it contain?

We can find out in a few different ways. The easiest (in Jupyter) is to run the `dir` function, which returns a list of strings, names that can go after a `.` after the module name.

If `dir(random)` includes the string `randint`, then we can use `random.randint` as a value or function, depending on what it is. I know that `random.randint` is a function, and we can then call

    random.randint(0, 100)

to get a random integer from 0-100.

If you want documentation showing not just the names, but what they do, then you can go to `docs.python.org` and look up the `random` module.

# Exercise: Guessing game

1. Have the computer choose a random integer from 0-100 using `random.randint` and assign to `number`.
2. Ask the user, repeatedly, to guess a number:
    - If it's too high, say "too high"
    - If it's too low, say "too low"
    - Otherwise, say "just right" and exit
3. If it's not right, then keep going, and ask the user to guess again.
4. When exiting the program, print not only the number, but how many guesses it took.

In [3]:
import random

number = random.randint(0, 100)
guess_counter = 0

while True:
    s = input('Guess: ').strip()

    if not s.isdigit():
        print('Not numeric; try again!')
        continue

    guess = int(s)
    guess_counter += 1

    if guess == number:
        print('You got it!')
        break
    elif guess < number:
        print('Too low!')
    else:
        print('Too high!')

print(f'Number was {number}, got it in {guess_counter} guesses.')        

Guess:  50


Too low!


Guess:  75


Too high!


Guess:  62


...

Too high!


Guess:  52


Too high!


Guess:  51


You got it!
Number was 51, got it in 7 guesses.


# Where did Python load `random` from?

We said `import random`, and we know that the `random` variable was assigned a module object, and that the module was populated from somewhere. But where? 

We can ask `random` to tell us about itself:

In [4]:
random

<module 'random' from '/Users/reuven/.pyenv/versions/3.13.5/lib/python3.13/random.py'>

How did saying `import random` get translated into loading the file in my `.pyenv` directory, deep down?

`sys.path` is the answer. The `sys` module is Python's runtime system. When Python runs, it consults `sys` all of the time. You have to `import sys` to use it, but that just defines the name; the module is loaded at runtime.

`sys.path` is a list of strings that Python consults to find modules we're loading.

In [5]:
import sys

sys.path

['/Users/reuven/.pyenv/versions/3.13.5/lib/python313.zip',
 '/Users/reuven/.pyenv/versions/3.13.5/lib/python3.13',
 '/Users/reuven/.pyenv/versions/3.13.5/lib/python3.13/lib-dynload',
 '',
 '/Users/reuven/.local/lib/python3.13/site-packages',
 '/Users/reuven/.pyenv/versions/3.13.5/lib/python3.13/site-packages']

In [6]:
random

<module 'random' from '/Users/reuven/.pyenv/versions/3.13.5/lib/python3.13/random.py'>

What if I want Python to look in other places? There are ways to do it. One way is with environment variables, set in your OS. 

# Can I just invoke `randint`?

What if I get tired of writing `random.randint` all of the time? I just want to write `randint`! Will that work?

In [7]:
randint(0, 100)

NameError: name 'randint' is not defined

# `from .. import`

The special `from .. import` syntax lets us import a module, and then instead of assigning the module's name as a variable, we get the function we named.

If I were to say

    from random import randint

then I wouldn't be able to say `random.randint`. That's beacuse the only thing we defined in the above line is one variable, `randint`. `random` isn't defined, and thus `random.randint` isn't avaiable.

Can you say both

    import random
    from random import randint

Yes, and sometimes I even encourage that.    

# What if I want to give the module an alias?

What if, instead of importing the module and getting its name dictated by its filename, I want to have the module variable defined as a different name.

For this, we have `import .. as`. This imports as per usual, but we get to define an alias for our module.

This is useful in two cases:

1. The name is super long and annoying
2. The name appears in more than one place, and we want to ensure that there aren't collisions.

In [11]:
import random as r

In [12]:
r.randint(0, 100)

57

# The fourth version: `from MODULE import NAME as ALIAS`

Then I can say


In [13]:
from random import randint as ri

# The four ways to use `import`:

1. `import MODNAME` -- the standard way to do things, which imports the entire module and defines `MODNAME` AS A VARIABLE
2. `IMPORT MODNAME AS ALIAS` -- just rename the variable that will be used for the module
3. `from MODNAME import NAME` -- don't define `MODNAME`, but do define `NAME`
4. `from MODNAME import NAME as ALIAS` -- rename the imported `NAME` as `ALIAS`.

# Importing more than one module

Each `import` needs to be on a line by itself.

If you want to import from `abcd` and `efgh` and `ijkl`, you have to say:

    import abcd
    import efgh
    import ijkl

If you're using `from .. import`, then you can import multiple names from the same module:

    from abcd import thing1, thing2
    

# Exercise: Another guessing game!

1. Define a list of strings, where each string represents the months of the year.
2. Use `random.choice`, a function in the `random` module, to select one of the months. However, you don't want to invoke it as `random.choice`, but rather as just `ch`, for short. You'll call `ch` on the list of month names, and get a random one.
3. Ask the user, repeatedly, to guess the month that was selected.
4. When they answer correctly, indicate how many times they guessed.

In [14]:
months = 'Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec'.split()

In [15]:
months

['Jan',
 'Feb',
 'Mar',
 'Apr',
 'May',
 'Jun',
 'Jul',
 'Aug',
 'Sep',
 'Oct',
 'Nov',
 'Dec']

In [16]:
# by using from .. import .. as
# - I imported the random module
# - I said that I want to define just the "choice" function from the module, not the module itself
# - but I want to alias "choice" to something else, "ch"

from random import choice as ch

In [20]:
ch(months)

'Dec'

In [21]:
from random import choice as ch

months = 'Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec'.split()
random_month = ch(months)
guess_counter = 0

while True:
    guess = input('Guess the month: ').strip().capitalize()[:3]
    guess_counter += 1

    if guess == random_month:
        print('You got it!')
        break

print(f'You got it in {guess_counter} guesses!')    

Guess the month:  Jul
Guess the month:  May
Guess the month:  Mar
Guess the month:  Jan
Guess the month:  Feb
Guess the month:  Apr
Guess the month:  Sep
Guess the month:  Oct
Guess the month:  Nov
Guess the month:  Dec
Guess the month:  Aug


You got it!
You got it in 11 guesses!


Many, *many* famous modules expect us to alias them when we import them:

- `import numpy as np`
- `import pandas as pd`
- `import matplotlib as plt`
- `from plotly import express as px`

# Next up

1. Write a simple module and `import` it
2. How do modules work?

# Writing a module

Writing a module is one of the easiest things you can do in Python! (Distributing it, packaging it up, etc., can be frustrating and confusing.) However, if you're writing a module that will be used in one particular program, then you can get away with just putting the module file inside of the program's directory. That's because `import` always looks first in the current directory.

What is a module?

- A file
- containing Python variable and function definitions
- with a `.py` suffix
- in a directory from which it can be loaded (in this case, the same as our program or our Jupyter notebook)

In [22]:
# right now, in the same directory as this Jupyter notebook, is an empty file
# called mymod.py

# if I say "import mymod", will it work?

import mymod

In [23]:
dir(mymod)  # show me all of the names defined on this module

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__']

# What is defined on an empty module?

Python did take our `import mymod` statement, look for `mymod.py` in the current directory, and even found it there! It then loaded the module's contents (very fast and easy